In [1]:
# Setup environ
import os 
from dotenv import load_dotenv
load_dotenv()

True

# Recall

ref : https://docs.langchain.com/docs/category/components

- Recall that Langchain comprises of 7 main components

![](asset/lang_components.png)

# Why LM alone is not enough

1. Data-aware: connect a language model to other sources of data

2. Agentic: allow a language model to interact with its environment

# 1. Schema

- The representation of theory you use to communicate with agent
- The structure of the prompt
- Langchain represent each type in form of object

There are 4 main types of that representation

## 1.1. Text

> Just a straightforward text prompt

In [2]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.0)

text = "If I love eating chocolate, which store name I need to go"
output = llm(text)
print(output)



If you love eating chocolate, you should go to a chocolate store, such as Godiva, Ghirardelli, Lindt, or Hershey's.


## 1.2. ChatMessage

> The idea behind this is to keep the memory of the previous state

> Langchain offers a better way instead of prompt like

"""
Instruction : You are the nutritionist who dedicated to help the people that lack of nutrition <br>

Here's the example <br>

User : I am flu, what should I eat<br>
AI : Try eating fruits with a rich of vitamin C<br>
User : {Input}<br>
AI : {Output}<br>
"""

> It comprises of 3 main components : 

- SystemChatMessage - Instruct the AI
- HumanMessage - What the user want to prompt
- AiMessage - What the Ai will prompt out  


In [3]:
from langchain.chat_models import ChatOpenAI
# These 3 are Chat schema
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Model type : Chat Models
chat = ChatOpenAI(temperature = 0.1)

In [ ]:
# Here is how we represent the prompt more structurally
chat_history = [
                SystemMessage(content="You are the nutritionist who dedicated to help the people that lack of nutrition"),
                HumanMessage(content="I am flu, what should I eat"),
                AIMessage(content="Try eating fruits with a rich of vitamin C"),
                HumanMessage(content="Which fruits are those")             
                ]

In [4]:
chat(chat_history)

# You can see the AIMessage come here

AIMessage(content='Some fruits that are rich in vitamin C include oranges, grapefruits, kiwis, strawberries, papayas, and pineapples. These fruits can help boost your immune system and fight off the flu. Additionally, you can also try drinking warm liquids like tea or soup to help soothe your throat and alleviate congestion.', additional_kwargs={}, example=False)

In [13]:
# Another example
# Here is how we represent the prompt more structurally
chat_history = [
                SystemMessage(content="You are a doctor intern who learn to investigate a patient"),
                AIMessage(content="Okay, how are you"),
                HumanMessage(content="I sneeze every 5 minutes")          
                ]

In [14]:
chat(chat_history)

# You can see the AIMessage come here

AIMessage(content="Okay, let's start by asking a few questions to better understand your symptoms. \n\n1. How long have you been experiencing these symptoms? \n2. Do you have any other symptoms such as a runny nose, itchy or watery eyes, or a sore throat? \n3. Have you been exposed to any allergens or irritants recently? \n4. Have you recently started taking any new medications or supplements? \n5. Have you had any recent changes in your environment or lifestyle? \n\nBased on your answers, I may recommend further testing or refer you to a specialist for further evaluation.", additional_kwargs={}, example=False)

## 1.3. Document

- So, the objective is to tell the document to use and its metadata

In [17]:
from langchain.schema import Document
import requests

document_url = 'https://icrrd.com/media/16-05-2021-080425Why-We-Sleep-Unlocking-the-Power-of-Sleep.pdf'
document_dest = 'source/why_we_sleep.pdf'

document = requests.get(document_url)
with open(document_dest, 'wb') as f:
    f.write(document.content)

print('imported successfully')

imported successfully


Sepcifically, it is used for filtering the specific documents in your database

In [18]:
Document(page_content= "This is a book about sleepinp",
        metadata={"book_name" : "Why we sleep by Matthew Walker",
                 "book_source" : document_dest,
                 "book_timestamp" : "11/10/2022"})

Document(page_content='This is a book about sleepinp', metadata={'book_name': 'Why we sleep by Matthew Walker', 'book_source': 'source/why_we_sleep.pdf', 'book_timestamp': '11/10/2022'})

## 1.4. Example

- The idea of N-shot prompt 

# 2. Model

## 2.1. LLMs

- text as an input and returns text as an output.

In [8]:
# % Already saw it

from langchain.llms import OpenAI
llm = OpenAI(temperature=0.0)

## 2.2. Chat models

- a list of ChatMessages as an input and returns a ChatMessage.

In [9]:
from langchain.chat_models import ChatOpenAI

# Model type : Chat Models
chat = ChatOpenAI(temperature = 0.1)

In [ ]:
# The prompt for chat model can be the chat_history we shown before

## 2.3. Text Embedding Model

OpenAI’s text embeddings measure the relatedness of text strings. Embeddings are commonly used for:

- Search (where results are ranked by relevance to a query string)
- Clustering (where text strings are grouped by similarity)
- Recommendations (where items with related text strings are recommended)
- Anomaly detection (where outliers with little relatedness are identified)
- Diversity measurement (where similarity distributions are analyzed)
- Classification (where text strings are classified by their most similar label)

In [10]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [11]:
txt_embedding = embeddings.embed_query("I love eating food") # Vector

print(len(txt_embedding))
print(txt_embedding[:5])

1536
[0.007248167414218187, -0.014703948982059956, -0.0025203716941177845, -0.0057032760232687, -0.026012800633907318]


These are the embedding vector for the query "I love eating food"

# 3. Prompt

- PromptValue

> The class representing an input to a model.

- Prompt Templates

> The class in charge of constructing a PromptValue.

- Example Selectors

> Often times it is useful to include examples in prompts. These examples can be hardcoded, but it is often more powerful if they are dynamically selected.

- Output Parsers

> Language models (and Chat Models) output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output Parsers are responsible for (1) instructing the model how output should be formatted, (2) parsing output into the desired formatting (including retrying if necessary).

## 3.1. Prompt value

In [20]:
prompt = """
I luv eating raw chicken

What's wrong with that
"""

## 3.2. Prompt Template

In [26]:
from langchain import OpenAI
from langchain import PromptTemplate

template = """

I really love {food}

What's wrong with that 
"""

# Declare the prompt template, and define the input var
prompt_template = PromptTemplate(input_variables = ["food"], template = template)
print(prompt_template)
# Demonstrate how to derive the final prompt
final_prompt = prompt_template.format(food = 'raw chicken')
print(final_prompt)

input_variables=['food'] output_parser=None partial_variables={} template="\n\nI really love {food}\n\nWhat's wrong with that \n" template_format='f-string' validate_template=True


I really love raw chicken

What's wrong with that 



In [27]:
llm = OpenAI(temperature=0)
llm(final_prompt)

'\nRaw chicken can contain harmful bacteria, such as salmonella, that can make you very sick if it is not cooked properly. It is important to always cook chicken thoroughly to an internal temperature of 165°F to ensure that any bacteria present is killed.'

## 3.3. Example Selectors

- Make the example (few shot learning) more relevant to the question example 
- In real world, we have a ton of examples, but the prompt cannot exceed the limit token, so to reduce the token, instead of showing them all examples, we pick the more appropriate example related to the question

Eg. 

```
Give the Location of an item usually found in

Example Input : driver
Example Output : car

Example Input : pilot
Example Output : plane

Input : student
Output : 
```


```
Give the Location of an item usually found in

Example Input : tree
Example Output : ground

Example Input : bird
Example Output : nest

Input : flower
Output : 
```

See the example is changed due to the input, the agent will try their best to pick the best example related to input

If the input prompt is flower, then the example pair that make the LLM do their best is 

(tree,ground), (bird,nest)

The reason behind this might be because (tree & flower) and (bird & flower) are closer than (driver & flower) and (pilot & flower)

Choosing the right index which is the most similar to the query,

In [28]:
# This will engineer the prompt first before entering to LLM
# Using the prompt template
# But the example is selected based on ...

from langchain import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS  # FAISS : lib that choose the right index which is the most similar to the query with the v. hgh speed

example_prompt = PromptTemplate(
    input_variables = ["input","output"],
    template = "Example Input : {input}, Example Output : {output}")

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"}, # This is the str format for example_prompt
    {"input": "pilot", "output": "plane"}, 
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [32]:
# There are 3 main things required here to select
#1.examples pair (examples)
#2.embedding function (OPENAI embedding)
#3.algorithm to output the index (FAISS) (imagine a billion vector, and have to sort the top 3 vector with highest similarity)

from langchain.embeddings import OpenAIEmbeddings

# Select the 
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # This is the list of examples available to select from. 
    OpenAIEmbeddings(),  # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    FAISS, # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    k=2 # This is the number of examples to produce.
)

In [33]:
example_selector

SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x16b146430>, k=2, example_keys=None, input_keys=None)

In [35]:
from langchain.prompts import FewShotPromptTemplate
#### Structure of few-shot-prompt #####
            # Instruction #
            # Example #
            # Question #

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector, # The object that will help select examples # 
    example_prompt=example_prompt,
    prefix="Give the location an item is usually found in",# Customizations that will be added to the top
    suffix="Input: {noun}\nOutput:", # and bottom of your prompt
    input_variables=["noun"],
)


FewShotPromptTemplate(input_variables=['noun'], output_parser=None, partial_variables={}, examples=None, example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x16b146430>, k=2, example_keys=None, input_keys=None), example_prompt=PromptTemplate(input_variables=['input', 'output'], output_parser=None, partial_variables={}, template='Example Input : {input}, Example Output : {output}', template_format='f-string', validate_template=True), suffix='Input: {noun}\nOutput:', example_separator='\n\n', prefix='Give the location an item is usually found in', template_format='f-string', validate_template=True)

In [39]:
# You can see the example is varied
my_noun = "student"
final_prompt = similar_prompt.format(noun=my_noun)
print(final_prompt)

Give the location an item is usually found in

Example Input : driver, Example Output : car

Example Input : pilot, Example Output : plane

Input: student
Output:


In [40]:
# Let's input the final prompt
llm(final_prompt)

' classroom'

# 3.4. Output Parser

- Want the output from the model to be JSON, or any strutural format, rather than string ? 
- langchain know how to prompt llm to make the output be in the desirable format


In [41]:
# The idea is the langchain will find the best {format} instruction for you

template = """
        You will be given a poorly formatted string from a user.
        Reformat it and make sure all the words are spelled correctly

        {format_instructions}

        % USER INPUT:
        {user_input}

        YOUR RESPONSE:
        """

In [42]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [44]:
# Print the output format
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [48]:
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

final_prompt = prompt.format(user_input = "i iove u")
print(final_prompt)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
i iove u

YOUR RESPONSE:



In [50]:
output = llm(final_prompt)
print(output)

```json
{
	"bad_string": "i iove u",
	"good_string": "I love you"
}
```


In [54]:
# Turn to json
output_parser.parse(output)

{'bad_string': 'i iove u', 'good_string': 'I love you'}

# 4. Indexs 

- Indexes refer to ways to structure documents so that LLMs can best interact with them
- It is impossible to input whole document to your LLM
- One possible technique they did is split the document into many documents by chunk size
- say your document has 10000 tokens, it an be splitted into 10, if you determine the chunk size as 1000
- and iteratively input those document to LLM
- However, this technique might not be common, because the LLM has to memorize and get distorted by irrelevant document
- So, instead of input all documents, we better input just the most relevant documents based on the user query that indexes which is called "retrieval" step. 

![](asset/index_langchain.png)
ref : https://www.youtube.com/watch?v=RIWbalZ7sTo

## 4.1. Document Loader

- Without this, how can you download the document. well, maybe use f.read(..)
- However, langchain offers us the better ways, since this Loader can interact with the rest of pipeline from langchain better


In [102]:
from langchain.document_loaders import PyPDFLoader

document_dest = 'source/why_we_sleep.pdf'

loader = PyPDFLoader(document_dest)

In [ ]:
documents = loader.load() # loader.load_and_split()

In [115]:
print(f"This pdf contains {len(documents)} pages")

This pdf contains 372 pages


## 4.2. Text splitter

- This is the part where we turn document into chunks

In [118]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS 

# Size is count by character not token
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)

texts = text_splitter.split_documents(documents)

In [119]:
print('number of documents (splitted) : ', len(texts))

number of documents (splitted) :  993


## 4.3. VectorStores

- Where can we store the embedding of the spllited document
- Here it is, if you have 993 documents, then 993 embedding vector were created and store in here
- We adopted FAISS which was vectorstore developed by Meta
- Its greatness is not only offering the place to store, but also how they quickly return the most relevant embedding vector regard of the query
- They find the index of the most relevant embedding vector

In [110]:
# We will use this function to emb each splitted document
embeddings = OpenAIEmbeddings()

# Embed and store in this
db = FAISS.from_documents(texts,embeddings)

## 4.4. Retrieval

- Every vectorstore have its own retrieval utility function 
- The goal here is to return the index of (splitted) document which has the highest similarity to the user query, so they can return the document that most relevant to query
- The top K documents is arbitrary, you may need to obtain top 1 or 5 documents regarding to query using search_kwarg

In [111]:
# Declare retriever object from database
# Get the top 5 documents

search_kwargs=dict(k=5)
retriever = db.as_retriever(search_kwargs = search_kwargs)

In [112]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x1254e8e50>, search_type='similarity', search_kwargs={'k': 5})

In [113]:
help(retriever.get_relevant_documents)

Help on method get_relevant_documents in module langchain.vectorstores.base:

get_relevant_documents(query: 'str') -> 'List[Document]' method of langchain.vectorstores.base.VectorStoreRetriever instance
    Get documents relevant for a query.
    
    Args:
        query: string to find relevant documents for
    
    Returns:
        List of relevant documents



In [114]:
retriever.get_relevant_documents('sleeping pill to help me sleep better',)

[Document(page_content='the report by stating that the eﬀect of current sleeping medications was\n“rather small and of questionable clinical importance.” Even the newest\nsleeping pill for insomnia, called suvorexant (brand name Belsomra), has\nproved minimally eﬀective, as we discussed in chapter 12. Future versions of\nsuch drugs may oﬀer meaningful sleep improvements, but for now the\nscientiﬁc data on prescription sleeping pills suggests that they may not be\nthe answer to returning sound sleep to those struggling to generate it on\ntheir own.\nSLEEPING PILLS— THE BAD, THE BAD, AND THE UGLY\nExisting prescription sleeping pills are minimally helpful, but are they\nharmful, even deadly? Numerous studies have something to say on this\npoint, yet much of the public remains unaware of their ﬁndings.\nNatural deep sleep, as we have previously learned, helps cement new\nmemory traces within the brain, part of which require the active\nstrengthening of connections between synapses that ma

# 5. Memory

- Instead of directly prompt after instruction template, we can even tell openai what we chatted to this before

In [136]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

history = ChatMessageHistory()
history.add_ai_message("Hi")
history.add_user_message("What is the capital of Thailand")
history.add_ai_message("It is Rome")
history.add_user_message("That's not true, I will give you one more chance")

In [137]:
history.messages
# You can observe that the history simply store the Chat schema with AI message and User mesage on our first chapter

[AIMessage(content='Hi', additional_kwargs={}, example=False),
 HumanMessage(content='What is the capital of Thailand', additional_kwargs={}, example=False),
 AIMessage(content='It is Rome', additional_kwargs={}, example=False),
 HumanMessage(content="That's not true, I will give you one more chance", additional_kwargs={}, example=False)]

In [138]:
ai_response = chat(history.messages)
ai_response # Just an AIMessage object, so w ecan append it to history

AIMessage(content='I apologize for the mistake. The capital of Thailand is Bangkok.', additional_kwargs={}, example=False)

In [139]:
history.add_ai_message(ai_response.content)
ai_response = chat(history.messages)
ai_response

AIMessage(content='How may I assist you today?', additional_kwargs={}, example=False)

# 6. Chains

- It's how we break the question into a series of questions. 


In [164]:
# Basic of chain (its input and output)
# You can see the output is json
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

In [141]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=1)

- Imagine you were asked to tell how to make the popular dish in Bangkok
- The very first thing gyou need to do is to search which dish is
- Then you will use that dish and find the ingredient and how to make that dish

We can summarize as this

input -> chain_1 -> output -> chain_2 -> output

We will use the outout fromt he chain 1 as input of chain 2

- In this case, we can use SimpleSequentialChain

- Note that each chain is LLMchain object

In [146]:
# Chain 1

template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [147]:
# Chain 2

template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [148]:
# Now, we can connect them together via simpleSequentialPipeline

overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [149]:
review = overall_chain.run("Bangkok")



> Entering new SimpleSequentialChain chain...
Pad Thai. This classic Thai dish is made with rice noodles, eggs, garlic, chili, peanuts and a sweet-sour sauce and is widely popular in Bangkok.

Ingredients:
- 2 cups rice noodles
- 2 eggs
- 4 cloves garlic
- 2 tbsp chili sauce
- 1/4 cup chopped peanuts
- 2 tbsp tamarind paste
- 2 tbsp brown sugar
- 2 tbsp fish sauce
- 2 tbsp lime juice

Instructions: 

1. Cook the rice noodles according to the package instructions.

2. In a separate pan, scramble 2 eggs and add in the minced garlic.

3. Mix in the chili sauce, peanuts, tamarind paste and brown sugar.

4. Add the noodles to the pan and combine all the ingredients together.

5. Finally, add the fish sauce and lime juice and stir fry until everything is evenly combined.

6. Serve the Pad Thai hot and enjoy!

> Finished chain.


# 7. Agents

- Sometimes the user query is so random, and predetermined chain is not flexible enough
- Agents is smart enough to figure out create their own chain, so you don't have to predetermined chain
- They can decided the tool, or source to use

### Agent Case 1 : Serpapi

In [155]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [151]:
import json

llm = OpenAI(temperature=0)
toolkit = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [152]:
response = agent({"input":"what was the first album of the" 
                    "band that Natalie Bergman is a part of?"})

print(json.dumps(response["intermediate_steps"], indent=2))



> Entering new AgentExecutor chain...
 I should try to find out what band Natalie Bergman is a part of.
Action: Search
Action Input: "Natalie Bergman band"
Observation: Natalie Bergman is an American singer-songwriter. She is one half of the duo Wild Belle, along with her brother Elliot Bergman. Her debut solo album, Mercy, was released on Third Man Records on May 7, 2021. She is based in Los Angeles.
Thought: I should search for the debut album of Wild Belle.
Action: Search
Action Input: "Wild Belle debut album"
Observation: Isles
Thought: I now know the final answer.
Final Answer: Isles is the debut album of Wild Belle, the band that Natalie Bergman is a part of.

> Finished chain.
[
  [
    [
      "Search",
      "Natalie Bergman band",
      " I should try to find out what band Natalie Bergman is a part of.\nAction: Search\nAction Input: \"Natalie Bergman band\""
    ],
    "Natalie Bergman is an American singer-songwriter. She is one half of the duo Wild Belle, along with her b

### Agent Case 2 : PdfIndex

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS 

# Size is count by character not token
def initialize_db(document_path):
    # Declare toolset
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
    embeddings = OpenAIEmbeddings()
    
    # Run
    loader = PyPDFLoader(document_path)
    texts = text_splitter.split_documents(documents)

    # Embed and store in this
    db = FAISS.from_documents(texts,embeddings)
    return db

In [162]:
document_path = 'source/why_we_sleep.pdf'
search_kwargss = {'k': 5}

db = initialize_db(document_path)
retriever = db.as_retriever(search_kwargs = search_kwargs)

In [163]:
query = 'which time we best sleep'
retriever.get_relevant_documents(query)

[Document(page_content='Appendix\nTwelve Tips for Healthy SleepI\n1.\xa0Stick to a sleep schedule. Go to bed and wake up at the same time each\nday. As creatures of habit, people have a hard time adjusting to changes in\nsleep patterns. Sleeping later on weekends won’t fully make up for a lack\nof sleep during the week and will make it harder to wake up early on\nMonday morning. Set an alarm for bedtime. Often we set an alarm for\nwhen it’s time to wake up but fail to do so for when it’s time to go to\nsleep. If there is only one piece of advice you remember and take from\nthese twelve tips, this should be it.\n2.\xa0Exercise is great, but not too late in the day. Try to exercise at least thirty\nminutes on most days but not later than two to three hours before your\nbedtime.\n3.\xa0Avoid caﬀeine and nicotine. Coﬀee, colas, certain teas, and chocolate\ncontain the stimulant caﬀeine, and its eﬀects can take as long as eight\nhours to wear oﬀ fully. \ue033erefore, a cup of coﬀee in the l

In [172]:
# There are 2 tools we allowed for this : LLM, Sleep_Database
# 1. LLM for general logics 
# 2. For specialization database about sleep

from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(input_variables = ['query'], template ='{query}')

# Initialize tools (LLM + Sleep_Database)
llm_chain = LLMChain(llm=llm,prompt=prompt)
llm_tool = Tool(name="Language model", func=llm_chain.run, description="Use this tool for general purpose queries and logics")
sleep_db_tool = Tool(name="Sleep Database", func=retriever.get_relevant_documents, description = "Use this tool when you want to search for the reference about sleeping")

tools = [llm_tool,sleep_db_tool]

# Initialize agent
zero_shot_agents = initialize_agent(agent='zero-shot-react-description',tools=tools,llm=llm,verbose=True,max_iterations=10)

In [169]:
zero_shot_agents("Which time is best to sleep")



> Entering new AgentExecutor chain...
 I need to find out what the research says about sleeping
Action: Sleep Database
Action Input: Best time to sleep
Observation: [Document(page_content='standard measure of sleep, estimated a range of 6 to 7.5 hours of this time\nwas spent asleep. \ue033e sleep opportunity that these tribespeople provide\nthemselves is therefore almost identical to what the National Sleep\nFoundation and the Centers for Disease Control and Prevention recommend\nfor all adult humans: 7 to 9 hours of time in bed.', metadata={'source': 'source/why_we_sleep.pdf', 'page': 249}), Document(page_content='Appendix\nTwelve Tips for Healthy SleepI\n1.\xa0Stick to a sleep schedule. Go to bed and wake up at the same time each\nday. As creatures of habit, people have a hard time adjusting to changes in\nsleep patterns. Sleeping later on weekends won’t fully make up for a lack\nof sleep during the week and will make it harder to wake up early on\nMonday morning. Set an alarm for 

{'input': 'Which time is best to sleep',
 'output': 'The best time to sleep is between 7-9 hours before your usual wake-up time.'}

In [178]:
from langchain.memory import ConversationBufferMemory

# Declare memory for it, it will lookup for variable chat_history when needed (error if not chat_history)
memory = ConversationBufferMemory(memory_key="chat_history")

# This is what we expected, but would be better if we do conversational instead
conversational_agents = initialize_agent(agent='conversational-react-description',
                                         tools=tools,llm=llm,
                                         memory = memory,
                                         verbose=True,
                                         max_iterations=10)


In [179]:
conversational_agents("what is the best time we should sleep")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Sleep Database
Action Input: best time to sleep
Observation: [Document(page_content='Appendix\nTwelve Tips for Healthy SleepI\n1.\xa0Stick to a sleep schedule. Go to bed and wake up at the same time each\nday. As creatures of habit, people have a hard time adjusting to changes in\nsleep patterns. Sleeping later on weekends won’t fully make up for a lack\nof sleep during the week and will make it harder to wake up early on\nMonday morning. Set an alarm for bedtime. Often we set an alarm for\nwhen it’s time to wake up but fail to do so for when it’s time to go to\nsleep. If there is only one piece of advice you remember and take from\nthese twelve tips, this should be it.\n2.\xa0Exercise is great, but not too late in the day. Try to exercise at least thirty\nminutes on most days but not later than two to three hours before your\nbedtime.\n3.\xa0Avoid caﬀeine and nicotine. Coﬀee, colas, certain teas, an

{'input': 'what is the best time we should sleep',
 'chat_history': '',
 'output': 'According to the National Sleep Foundation and the Centers for Disease Control and Prevention, adults should get 7 to 9 hours of sleep each night. It is best to stick to a regular sleep schedule and avoid caffeine and nicotine before bed. Exercise should be done at least 30 minutes before bed, and it is important to get an hour of exposure to morning sunlight and turn down the lights before bedtime.'}

In [180]:
conversational_agents("Why 7-9 hours is the best ?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Seven to nine hours of sleep is recommended because it is the amount of time needed for the body to go through all of the stages of sleep, including deep sleep and REM sleep. Deep sleep helps to restore the body and mind, while REM sleep helps to consolidate memories and process emotions. Getting enough sleep is important for physical and mental health, and can help to improve concentration, mood, and overall wellbeing.

> Finished chain.


{'input': 'Why 7-9 hours is the best ?',
 'chat_history': 'Human: what is the best time we should sleep\nAI: According to the National Sleep Foundation and the Centers for Disease Control and Prevention, adults should get 7 to 9 hours of sleep each night. It is best to stick to a regular sleep schedule and avoid caffeine and nicotine before bed. Exercise should be done at least 30 minutes before bed, and it is important to get an hour of exposure to morning sunlight and turn down the lights before bedtime.',
 'output': 'Seven to nine hours of sleep is recommended because it is the amount of time needed for the body to go through all of the stages of sleep, including deep sleep and REM sleep. Deep sleep helps to restore the body and mind, while REM sleep helps to consolidate memories and process emotions. Getting enough sleep is important for physical and mental health, and can help to improve concentration, mood, and overall wellbeing.'}

In [181]:
conversational_agents("Why 7-9 hours is the best ? Please use sleep database tool")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Sleep Database
Action Input: Why 7-9 hours is the best?
Observation: [Document(page_content='beings do not, after all, need a full eight hours of sleep, some suggesting we\ncan survive just ﬁne on six hours or less. For example, the headline of one\nprominent US newspaper read:\n“Sleep Study on Modern-Day Hunter-Gatherers Dispels Notion \ue033at We’re\nWired to Need 8 Hours a Day.”\nOthers started out with the already incorrect assumption that modern\nsocieties need only seven hours of sleep, and then questioned whether we\neven need that much: “Do We Really Need to Sleep 7 Hours a Night?”\nHow can such prestigious and well-respected entities reach these\nconclusions, especially after the science that I have presented in this\nchapter? Let us carefully reevaluate the ﬁndings, and see if we still arrive at\nthe same conclusion.\nFirst, when you read the paper, you will learn that the tribespeople were

{'input': 'Why 7-9 hours is the best ? Please use sleep database tool',
 'chat_history': 'Human: what is the best time we should sleep\nAI: According to the National Sleep Foundation and the Centers for Disease Control and Prevention, adults should get 7 to 9 hours of sleep each night. It is best to stick to a regular sleep schedule and avoid caffeine and nicotine before bed. Exercise should be done at least 30 minutes before bed, and it is important to get an hour of exposure to morning sunlight and turn down the lights before bedtime.\nHuman: Why 7-9 hours is the best ?\nAI: Seven to nine hours of sleep is recommended because it is the amount of time needed for the body to go through all of the stages of sleep, including deep sleep and REM sleep. Deep sleep helps to restore the body and mind, while REM sleep helps to consolidate memories and process emotions. Getting enough sleep is important for physical and mental health, and can help to improve concentration, mood, and overall wel